In [7]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient
import requests

In [8]:
# Keys
PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]

Serverless index from [this dataset](https://huggingface.co/datasets/Cohere/wikipedia-22-12).

In [9]:
# Init
pinecone = PineconeClient(api_key=PINECONE_API_KEY,
                         environment=PINECONE_ENVIRONMENT)

embeddings = CohereEmbeddings(model="multilingual-22-12")
vectorstore = Pinecone.from_existing_index(index_name=PINECONE_INDEX_NAME, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [10]:
# RAG prompt
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG
model = ChatOpenAI(temperature=0, 
                   model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [11]:
chain.invoke("what is film noir?")

'Film noir is a genre of film that is characterized by a set of attributes that provoke ongoing debate regarding its definition. According to French critics, including Étienne Chaumeton, in their 1955 book "Panorama du film noir américain 1941–1953" ("A Panorama of American Film Noir"), film noir can be described as oneiric (dreamlike), strange, erotic, ambivalent, and cruel, although not every film noir necessarily embodies all these attributes to the same degree. The genre is known for its complexity, with films that may feature crime and violence, complex characters and plot-lines, mystery, and moral ambivalence.\n\nFilm noir is also recognized for its visual style, which may include the use of chiaroscuro lighting techniques, and is often associated with a bleak societal perspective, offering a critique on global capitalism and consumerism. This is particularly evident in the neon-noir sub-genre, which emphasizes the socio-critical aspects of film noir and often includes long shots

Extract full wiki page.

In [12]:
def fetch_wikipedia_page(id):
    url = f"https://en.wikipedia.org/w/api.php?action=query&prop=extracts&format=json&pageids={id}"
    response = requests.get(url)
    data = response.json()
    page_content = list(data['query']['pages'].values())[0]['extract']
    return page_content

def fetch_url(x):
    urls = [doc.metadata['url'] for doc in x['context']]
    ids = [url.split('=')[-1] for url in urls]
    # First 32k tokens
    contents = [fetch_wikipedia_page(id)[:32000] for id in ids]    
    return {"context": contents, "question": x["question"]}

# RAG
model = ChatOpenAI(temperature=0, 
                   model="gpt-4-1106-preview")

chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | RunnableLambda(fetch_url)  
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
chain.invoke("what is film noir?")

'Film noir is a cinematic term used primarily to describe stylized Hollywood crime dramas, particularly those that emphasize cynical attitudes and motivations. The classic period of American film noir is generally regarded as the 1940s and 1950s. Film noir of this era is associated with a low-key, black-and-white visual style that has roots in German Expressionist cinematography. It encompasses a range of plots and central figures, including private investigators, plainclothes police officers, aging boxers, hapless grifters, law-abiding citizens lured into a life of crime, femme fatales, or victims of circumstance.'